## Imports

In [ ]:
import os
import cv2
import torch
import shutil
import random
import numpy as np

from glob import glob
from monai.transforms import LoadImage
from torchvision import transforms
from pytorch_fid import fid_score
from pytorch_msssim import ssim

## Divide Real Images in Half

In [ ]:
def split_images(src_folder, dest_folder1, dest_folder2):
    # Create destination folders if they don't exist
    os.makedirs(dest_folder1, exist_ok=True)
    os.makedirs(dest_folder2, exist_ok=True)
    
    # Get list of image files in the source folder
    image_files = glob(os.path.join(src_folder, '*.png'))
    
    # Split the image files list in half
    midpoint = len(image_files) // 2
    first_half = image_files[:midpoint]
    second_half = image_files[midpoint:]
    
    # Copy first half to the first destination folder
    for image_file in first_half:
        shutil.copy(image_file, dest_folder1)
    for image_file in second_half:
        shutil.copy(image_file, dest_folder2)
        
    print(f'{len(first_half)} images copied to {dest_folder1}')
    print(f'{len(second_half)} images copied to {dest_folder2}')

# Example usage:
src_folder = './real_images/275'
dest_folder1 = './real_images/275_1'
dest_folder2 = './real_images/275_2'

split_images(src_folder, dest_folder1, dest_folder2)

## FID Calculation

In [ ]:
def calculate_fid(path1, path2, batch_size=60, device='cuda', dims=768, label=None):
    fid = fid_score.calculate_fid_given_paths([path1, path2], batch_size=batch_size, device=device, dims=dims)
    if label:
        print(f"FID {label}: {fid}")
    else:
        print(f"FID between {os.path.basename(path1)} and {os.path.basename(path2)}: {fid}")
    return fid

# Real vs Real
real_base = "./real_images/275"
real_1 = "./real_images/275_1"
real_2 = "./real_images/275_2"
calculate_fid(real_1, real_2, label="real vs real")

# Real vs Generated (guidance scale 별)
guidance_scales = [7.0, 5.0, 3.0, 1.0]
for scale in guidance_scales:
    generated_path = f"./generated_images/{scale}"
    calculate_fid(real_base, generated_path, label=f"{scale} real vs generated")

## SSIM Calculation

In [ ]:
def load_images_from_folder(folder, device='cuda'):
    images = [] 
    transform = transforms.Compose([
        transforms.ToTensor()  # Convert image to tensor [0,1] range
    ])
    
    for filename in sorted(os.listdir(folder)):
        if filename.endswith('.png'):
            file_path = os.path.join(folder, filename)
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE) 
            img = transform(img).unsqueeze(0).to(device)
            images.append(img)
    return images

def calculate_ssim(images):
    ssim_values = []
    image_count = len(images)
    
    # Generate all possible pairs without replacement
    indices = list(range(image_count))
    random.shuffle(indices)

    for i in range(image_count):
        for j in range(i + 1, image_count):
            img1 = images[indices[i]]
            img2 = images[indices[j]]

            # Compute SSIM
            ssim_value = ssim(img1, img2, data_range=1.0)
            ssim_values.append(ssim_value.item())

    return ssim_values

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def calculate_and_print_ssim(path, label=None, device=None):
    images = load_images_from_folder(path, device=device)
    ssim_values = calculate_ssim(images)
    mean_ssim = np.mean(ssim_values)
    if label:
        print(f"Average SSIM {label}: {mean_ssim}")
    else:
        print(f"Average SSIM for {path}: {mean_ssim}")
    return mean_ssim

# Real vs Real
real_path = "./real_images/275"
calculate_and_print_ssim(real_path, label="real vs real", device=device)

# Generated vs Generated (guidance scale 별)
guidance_scales = [7.0, 5.0, 3.0, 1.0]
for scale in guidance_scales:
    gen_path = f"./generated_images/{scale}"
    calculate_and_print_ssim(gen_path, label=f"{scale} generated vs generated")
